# **Improving Employee Retention By Predicting Employee Attrition Using Machine Learning**

---
# Business Understanding
---

Human resources (HR) represent the fundamental assets that require effective management for the company to achieve its business goals efficiently. In this context, we are addressing a challenge related to human resources within the company. Our primary objective is to understand how to retain employees within the current organization, recognizing that turnover incurs increased costs for employee recruitment and training, especially for those who have recently joined. Identifying the key factors contributing to employee dissatisfaction allows the company to proactively address these issues by implementing targeted programs that directly tackle employee concerns.

---
# Import Library
---

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import plotly.graph_objects as go

In [2]:
print (pd.options.display.max_columns)
pd.options.display.max_columns = 100
print (pd.options.display.max_columns)

20
100


---
# Load Data
---

In [3]:
df = pd.read_excel('dataset/Improving Employee Retention by Predicting Employee Attrition Using Machine Learning.xlsx')
df

,Username,EnterpriseID,StatusPernikahan,JenisKelamin,StatusKepegawaian,Pekerjaan,JenjangKarir,PerformancePegawai,AsalDaerah,HiringPlatform,SkorSurveyEngagement,SkorKepuasanPegawai,JumlahKeikutsertaanProjek,JumlahKeterlambatanSebulanTerakhir,JumlahKetidakhadiran,NomorHP,Email,TingkatPendidikan,PernahBekerja,IkutProgramLOP,AlasanResign,TanggalLahir,TanggalHiring,TanggalPenilaianKaryawan,TanggalResign
0,spiritedPorpoise3,111065,Belum_menikah,Pria,Outsource,Software Engineer (Back End),Freshgraduate_program,Sangat_bagus,Jakarta Timur,Employee_Referral,4,4.0,0.0,0.0,9.0,+6282232522xxx,spiritedPorpoise3135@yahoo.com,Magister,1,1.0,masih_bekerja,1972-07-01,2011-01-10,2016-2-15,-
1,jealousGelding2,106080,Belum_menikah,Pria,FullTime,Data Analyst,Freshgraduate_program,Sangat_kurang,Jakarta Utara,Website,4,4.0,4.0,0.0,3.0,+6281270745xxx,jealousGelding2239@yahoo.com,Sarjana,1,1.0,toxic_culture,1984-04-26,2014-01-06,2020-1-17,2018-6-16
2,pluckyMuesli3,106452,Menikah,Pria,FullTime,Software Engineer (Front End),Freshgraduate_program,Bagus,Jakarta Timur,Indeed,4,3.0,0.0,0.0,11.0,+6281346215xxx,pluckyMuesli3961@icloud.com,Magister,1,1.0,jam_kerja,1974-01-07,2011-01-10,2016-01-10,2014-9-24
3,stressedTruffle1,106325,Belum_menikah,Pria,Outsource,Software Engineer (Front End),Freshgraduate_program,Bagus,Jakarta Pusat,LinkedIn,3,3.0,0.0,4.0,6.0,+6283233846xxx,stressedTruffle1406@hotmail.com,Sarjana,1,0.0,masih_bekerja,1979-11-24,2014-2-17,2020-02-04,-
4,shyTermite7,111171,Belum_menikah,Wanita,FullTime,Product Manager,Freshgraduate_program,Bagus,Jakarta Timur,LinkedIn,3,3.0,0.0,0.0,11.0,+6287883263xxx,shyTermite7149@gmail.com,Sarjana,1,0.0,ganti_karir,1974-11-07,2013-11-11,2020-1-22,2018-09-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,dopeySheep0,106034,Belum_menikah,Wanita,FullTime,Data Engineer,Mid_level,Bagus,Jakarta Pusat,Google_Search,2,5.0,0.0,0.0,16.0,+6283343465xxx,dopeySheep0297@icloud.com,Sarjana,1,NaN,NaN,1973-12-08,2011-9-26,2016-03-01,-
283,yearningPorpoise4,106254,-,Wanita,FullTime,Product Design (UI & UX),Freshgraduate_program,Biasa,Jakarta Timur,LinkedIn,4,5.0,0.0,0.0,11.0,+6281222782xxx,yearningPorpoise4421@yahoo.com,Sarjana,1,NaN,jam_kerja,1974-12-01,2013-5-13,2020-1-28,2017-11-15
284,murkySausage9,110433,Menikah,Wanita,FullTime,Software Engineer (Front End),Senior_level,Biasa,Jakarta Pusat,Diversity_Job_Fair,2,5.0,0.0,0.0,17.0,+6285821950xxx,murkySausage9688@yahoo.com,Sarjana,1,1.0,ganti_karir,1969-10-30,2013-11-11,2020-1-21,2018-06-08
285,truthfulMoth4,110744,Belum_menikah,Pria,FullTime,Software Engineer (Android),Mid_level,Bagus,Jakarta Utara,Google_Search,4,5.0,0.0,0.0,20.0,+6287787391xxx,truthfulMoth4663@yahoo.com,Sarjana,1,0.0,kejelasan_karir,1981-10-01,2011-5-16,2014-04-05,2018-04-01


---
# Stage 1: Data Preprocessing
---

## Univariate Analysis

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 25 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Username                            287 non-null    object 
 1   EnterpriseID                        287 non-null    int64  
 2   StatusPernikahan                    287 non-null    object 
 3   JenisKelamin                        287 non-null    object 
 4   StatusKepegawaian                   287 non-null    object 
 5   Pekerjaan                           287 non-null    object 
 6   JenjangKarir                        287 non-null    object 
 7   PerformancePegawai                  287 non-null    object 
 8   AsalDaerah                          287 non-null    object 
 9   HiringPlatform                      287 non-null    object 
 10  SkorSurveyEngagement                287 non-null    int64  
 11  SkorKepuasanPegawai                 282 non-n

According to the data frame information, several features exhibit missing values, and there is a need to adjust the data types for ease of processing. Here are the details:

**Features with Missing Values:**
- SkorKepuasanPegawai, JumlahKeikutsertaanProjek, JumlahKetidakhadiran, IkutProgramLOP, AlasanResign contain missing values.

**Features Requiring Datatype Change:**
- TanggalLahir, TanggalHiring, TanggalPenilaianKaryawan, TanggalResign will be converted to datetime datatype for better processing.

In [5]:
categorical = ['StatusPernikahan', 'JenisKelamin', 'StatusKepegawaian', 'Pekerjaan', 'JenjangKarir', 'PerformancePegawai', 'AsalDaerah', 'HiringPlatform', 'TingkatPendidikan', 'PernahBekerja', 'AlasanResign']
numerical = ['SkorSurveyEngagement', 'SkorKepuasanPegawai', 'JumlahKeikutsertaanProjek', 'JumlahKeterlambatanSebulanTerakhir', 'JumlahKetidakhadiran', 'IkutProgramLOP']

In [6]:
df[categorical].describe()

,StatusPernikahan,JenisKelamin,StatusKepegawaian,Pekerjaan,JenjangKarir,PerformancePegawai,AsalDaerah,HiringPlatform,TingkatPendidikan,PernahBekerja,AlasanResign
count,287,287,287,287,287,287,287,287,287,287,221
unique,5,2,3,14,3,5,5,9,3,2,11
top,Belum_menikah,Wanita,FullTime,Software Engineer (Back End),Freshgraduate_program,Biasa,Jakarta Pusat,Indeed,Sarjana,1,masih_bekerja
freq,132,167,217,109,169,85,72,85,176,286,132


There are several categorical features, namely StatusKepegawaian, JenjangKarir, and PernahBekerja, that exhibit a significant number of top-category differences compared to others. These features warrant further analysis to determine whether this feature is good or not.

In [7]:
for i in categorical:
    print('=======================================================================================================')
    print(i, ':', len(df[i].unique()))
    print(i, ':', df[i].unique())

StatusPernikahan : 5
StatusPernikahan : ['Belum_menikah' 'Menikah' 'Bercerai' 'Lainnya' '-']
JenisKelamin : 2
JenisKelamin : ['Pria' 'Wanita']
StatusKepegawaian : 3
StatusKepegawaian : ['Outsource' 'FullTime' 'Internship']
Pekerjaan : 14
Pekerjaan : ['Software Engineer (Back End)' 'Data Analyst'
 'Software Engineer (Front End)' 'Product Manager'
 'Software Engineer (Android)' 'Scrum Master'
 'Product Design (UX Researcher)' 'Product Design (UI & UX)'
 'Digital Product Manager' 'Data Engineer' 'Software Engineer (iOS)'
 'DevOps Engineer' 'Software Architect' 'Machine Learning Engineer']
JenjangKarir : 3
JenjangKarir : ['Freshgraduate_program' 'Senior_level' 'Mid_level']
PerformancePegawai : 5
PerformancePegawai : ['Sangat_bagus' 'Sangat_kurang' 'Bagus' 'Biasa' 'Kurang']
AsalDaerah : 5
AsalDaerah : ['Jakarta Timur' 'Jakarta Utara' 'Jakarta Pusat' 'Jakarta Selatan'
 'Jakarta Barat']
HiringPlatform : 9
HiringPlatform : ['Employee_Referral' 'Website' 'Indeed' 'LinkedIn' 'CareerBuilder'
 'Di

Based on the unique values in categorical features, the following improvements will be made to the data:

- **StatusPernikahan:**
  - Unique values '-' and 'lainnya' will be changed to 'unknown' for standardization.

- **PernahBekerja:**
  - The feature will be dropped as the unique values do not provide meaningful information.


In [8]:
df[numerical].describe()

,SkorSurveyEngagement,SkorKepuasanPegawai,JumlahKeikutsertaanProjek,JumlahKeterlambatanSebulanTerakhir,JumlahKetidakhadiran,IkutProgramLOP
count,287.000000,282.000000,284.000000,286.000000,281.000000,29.000000
mean,3.101045,3.904255,1.179577,0.412587,10.448399,0.517241
std,0.836388,0.913355,2.294441,1.275016,6.902252,0.508548
min,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,3.000000,3.000000,0.000000,0.000000,5.000000,0.000000
50%,3.000000,4.000000,0.000000,0.000000,10.000000,1.000000
75%,4.000000,5.000000,0.000000,0.000000,15.000000,1.000000
max,5.000000,5.000000,8.000000,6.000000,55.000000,1.000000


Based on the analysis of numerical features, all features exhibit normal values, and although some features display skewness, no corrective actions will be taken. This decision suggests that the overall distribution and characteristics of the numerical features are considered acceptable, and no adjustments are deemed necessary at this point.

In [9]:
# Changing datatype into datetime
df['TanggalLahir'] = pd.to_datetime(df['TanggalLahir'], format='%Y-%m-%d')
df['TanggalHiring'] = pd.to_datetime(df['TanggalHiring'], format='%Y-%m-%d')
df['TanggalPenilaianKaryawan'] = pd.to_datetime(df['TanggalPenilaianKaryawan'], format='%Y-%m-%d')
df['TanggalResign'].replace('-', np.nan, inplace=True)
df['TanggalResign'] = pd.to_datetime(df['TanggalResign'], format='%Y-%m-%d')

# Change value in StatusPernikahan
df['StatusPernikahan'].replace(['-', 'Lainnya'], 'unknown', inplace=True)

# Drop not meaningful columns
df.drop(['PernahBekerja'], axis = 1, inplace = True)

## Handling Missing Value

In [10]:
df.isnull().sum()

Username                                0
EnterpriseID                            0
StatusPernikahan                        0
JenisKelamin                            0
StatusKepegawaian                       0
Pekerjaan                               0
JenjangKarir                            0
PerformancePegawai                      0
AsalDaerah                              0
HiringPlatform                          0
SkorSurveyEngagement                    0
SkorKepuasanPegawai                     5
JumlahKeikutsertaanProjek               3
JumlahKeterlambatanSebulanTerakhir      1
JumlahKetidakhadiran                    6
NomorHP                                 0
Email                                   0
TingkatPendidikan                       0
IkutProgramLOP                        258
AlasanResign                           66
TanggalLahir                            0
TanggalHiring                           0
TanggalPenilaianKaryawan                0
TanggalResign                     

Based on the quantity of missing values in each column, the following actions will be taken:
- 'IkutProgramLOP' will be dropped due to the substantial number of missing values.
- 'TanggalResign' is set to be replaced with 'Masih Bekerja'.
- 'AlasanResign' is set to be replaced with 'unknown'.
- 'SkorKepuasanPegawai', 'JumlahKeikutsertaanProjek', 'JumlahKeterlambatanSebulanTerakhir', and 'JumlahKetidakhadiran' will be replaced with the median. Given the limited data, this approach is suggested to minimize data loss.

In [11]:
df.drop(['IkutProgramLOP'], axis = 1, inplace = True)
df["AlasanResign"].fillna("unknown", inplace = True)
df['SkorKepuasanPegawai'].fillna(df['SkorKepuasanPegawai'].median(), inplace = True)
df['JumlahKeikutsertaanProjek'].fillna(df['JumlahKeikutsertaanProjek'].median(), inplace = True)
df['JumlahKeterlambatanSebulanTerakhir'].fillna(df['JumlahKeterlambatanSebulanTerakhir'].median(), inplace = True)
df['JumlahKetidakhadiran'].fillna(df['JumlahKetidakhadiran'].median(), inplace = True)

## Handling Duplicated Data

In [12]:
df.duplicated().sum()

0

There is no duplicate value.

## Feature Engineering

Date data can be challenging to use directly in machine learning models, so a transformation will be applied to make it more useful. Specifically, the date data will be extracted to create two new features:

1. **Umur_pd_Penilaian:**
   - This feature will represent the age of the employee at the time of the 'penilaian karyawan'.

2. **Months_Hiring_Penilaian (Months_between_Hiring_and_Assessment):**
   - This feature will represent the number of Months between the hiring date and the 'penilaian karyawan' date.

3. **Tahun_Hiring**
   - This feature is obtained from TanggalHiring

4. **Tahun_Resign**
   - This feature is obtained from TanggalResign
   
These new features aim to provide more meaningful and model-friendly representations of the time-related information in the dataset.

In [13]:
df['Umur_pd_Penilaian'] = df['TanggalPenilaianKaryawan'].dt.year - df['TanggalLahir'].dt.year
df['Months_Hiring_Penilaian'] = ((df['TanggalPenilaianKaryawan']-df['TanggalHiring'])/np.timedelta64(1, 'M'))
df['Tahun_Hiring'] = df['TanggalHiring'].dt.year
df['Tahun_Resign'] = df['TanggalResign'].dt.year
df['Umur_pd_Penilaian'] = df['Umur_pd_Penilaian'].astype(int)
df['Months_Hiring_Penilaian'] = df['Months_Hiring_Penilaian'].astype(int)

The dataset will be done for deeper analysis, steps like feature encoding, scaling, splitting, and similar processes will be deferred to a later stage.

---
# Stage 2: Annual Report On Employee Number Changes
---

- Number of Hiring and Resign employee based on year

In [14]:
df_hiring = df.groupby(['Tahun_Hiring']).agg({
    'Tahun_Hiring' : ['count']
}).reset_index()
df_hiring.columns = ['Tahun', 'Jumlah Hiring']
df_hiring

,Tahun,Jumlah Hiring
0,2006,1
1,2007,2
2,2008,2
3,2009,7
4,2010,8
5,2011,76
6,2012,41
7,2013,43
8,2014,56
9,2015,31


In [15]:
df_resign = df.groupby(['Tahun_Resign']).agg({
    'Tahun_Resign' : ['count']
}).reset_index()
df_resign.columns = ['Tahun', 'Jumlah Resign']
df_resign['Tahun'] = df_resign['Tahun'].astype(int)
df_resign

,Tahun,Jumlah Resign
0,2013,5
1,2014,12
2,2015,8
3,2016,8
4,2017,19
5,2018,26
6,2019,5
7,2020,6


- Join Table Hiring and Resign

In [16]:
df_Number_Changes = pd.merge(df_hiring, df_resign, on='Tahun', how='outer')
df_Number_Changes.fillna(0, inplace=True)
df_Number_Changes

,Tahun,Jumlah Hiring,Jumlah Resign
0,2006,1.0,0.0
1,2007,2.0,0.0
2,2008,2.0,0.0
3,2009,7.0,0.0
4,2010,8.0,0.0
5,2011,76.0,0.0
6,2012,41.0,0.0
7,2013,43.0,5.0
8,2014,56.0,12.0
9,2015,31.0,8.0


- The total number of employees who resigned, those who still remain, and the changes each year.

In [17]:
df_Number_Changes['Pertumbuhan Karyawan'] = df_Number_Changes.apply(lambda x: x['Jumlah Hiring'] - x['Jumlah Resign'], axis=1)
listPertumbuhan = list(df_Number_Changes['Pertumbuhan Karyawan'])
total = [listPertumbuhan[0]]
for i in range(1, len(listPertumbuhan)):
    total.append(total[i-1]+listPertumbuhan[i])
df_Number_Changes['Total Karyawan'] = total
df_Number_Changes

,Tahun,Jumlah Hiring,Jumlah Resign,Pertumbuhan Karyawan,Total Karyawan
0,2006,1.0,0.0,1.0,1.0
1,2007,2.0,0.0,2.0,3.0
2,2008,2.0,0.0,2.0,5.0
3,2009,7.0,0.0,7.0,12.0
4,2010,8.0,0.0,8.0,20.0
5,2011,76.0,0.0,76.0,96.0
6,2012,41.0,0.0,41.0,137.0
7,2013,43.0,5.0,38.0,175.0
8,2014,56.0,12.0,44.0,219.0
9,2015,31.0,8.0,23.0,242.0


- Plot

In [36]:
fig = go.Figure()
fig.add_trace(go.Waterfall(x = df_Number_Changes["Tahun"],
                          y = df_Number_Changes["Pertumbuhan Karyawan"],
                        #   measure = df["measure"].tolist(),
                          base = 0,   #by default
                           connector = {"line":{"dash":"dot"}}, #get dotted line as connector
                           textposition = "outside",
                           text = df_Number_Changes["Pertumbuhan Karyawan"].tolist(),
                           orientation = "v"  #by default
                          ))
fig.update_layout(title = "Pertumbuhan Karyawan Tiap Tahun")
fig.update_xaxes(range = (2006, 2020), title = "Tahun")
fig.update_yaxes(range = (0, 300), title = "Pertumbuhan Karyawan")
fig.update_layout(
    margin=dict(l=50, r=30, t=50, b=50),
)
fig.show()

From the plot, the employee count has shown an increase from 2006 to 2016, with a significant increase observed between 2011 and 2015, suggesting a period of significant expansion for the company. However, a decrease in the number of employees is evident from 2017 to 2020, a decrease that warrants special attention as it may impact the company's overall performance.